Field Interpolation
==============

Given a set of samples of an unkown function, _estimate_ a function $f$


In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(style='darkgrid', context='talk', palette='Pastel1')


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]

$$f(0) = -1$$
$$f(1) = 0.2$$
$$f(2) = 0.9$$
$$f(3) = 2.1$$

Estimate a line, $f(x) = kx + m$, using linear last squares

$$-1 = k*0 + m$$
$$0.2 = k*1 + m$$
$$0.9 = k*2 + m$$
$$2.1 = k*3 + m$$

Or just

$$
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
2 & 1 \\
3 & 1 \\
\end{bmatrix}
\begin{bmatrix}
k & m
\end{bmatrix}
= 
\begin{bmatrix}
-1 \\ 0.2 \\ 0.9 \\ 2.1 \\
\end{bmatrix}$$
$$Ax = b$$

In [ ]:
import numpy as np

# sampled points
x = np.array([0, 1, 2, 3])
y = np.array([-1, 0.2, 0.9, 2.1])

A = np.vstack([x, np.ones(len(x))]).T
k, m = np.linalg.lstsq(A, y, rcond=None)[0]

In [ ]:
plt.plot(x, y, 'o', label='Original data')
plt.plot(x, k * x + m, label='Fitted line')
plt.legend()
plt.show()